# Chatbot Application

## Setup

### Install Dependencies

In [ ]:
import os
os.environ['HF_HOME'] = 'E:/Koding dan Kecerdasan Artificial/models'
os.environ['TRANSFORMERS_CACHE'] = 'E:/Koding dan Kecerdasan Artificial/models'

In [1]:
# Install uv package manager if not already installed
import subprocess

try:
    subprocess.run(['uv', '--version'], capture_output=True, check=True)
    print("✓ uv is already installed")
except (subprocess.CalledProcessError, FileNotFoundError):
    print("Installing uv...")
    %pip install uv
    print("✓ uv installed successfully")
    print("Initializing uv...")
    %uv init .
    print("✓ uv initialized successfully")

✓ uv is already installed


In [2]:
# Environment Detection and Setup
try:
    import google.colab # Check if running in Google Colab
    IN_COLAB = True
    print("Running in Google Colab")
    cache_dir = None  # Colab uses default HF cache
except:
    IN_COLAB = False
    print("Running locally")
    # Use local models directory
    import os
    models_cache_dir = "./models"
    os.makedirs(models_cache_dir, exist_ok=True)

# Install dependencies with uv (works in both environments)
!uv add gradio torch transformers accelerate bitsandbytes compressed-tensors --quiet

import torch
print(f"✓ PyTorch version: {torch.__version__}")
print(f"✓ CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✓ GPU: {torch.cuda.get_device_name(0)}")

Running in Google Colab
error: No `pyproject.toml` found in current directory or any parent directory
✓ PyTorch version: 2.8.0+cu126
✓ CUDA available: True
✓ GPU: Tesla T4


### Import Libraries

In [3]:
# Gradio Interface
import gradio as gr

# Hugging Face
from transformers import AutoModelForCausalLM, AutoTokenizer

### Setup Models

In [4]:
# Setup models for each task

# Text Generation
model_repo = "Qwen/Qwen3-0.6B" # For huggingface use this directly
model_name = "Qwen/Qwen3-0.6B" # For llama.cpp

## Run the Application

### Hugging Face

In [5]:
# Load the model
import torch
import gc

# Clear any existing model from memory
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()


print("Loading tokenizer and model...")
tokenizer = AutoTokenizer.from_pretrained(
    model_repo, 
    trust_remote_code=True,
    cache_dir=models_cache_dir
)
model = AutoModelForCausalLM.from_pretrained(
    model_repo,
    dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto",
    trust_remote_code=True,
    cache_dir=models_cache_dir
)

print("✓ Model loaded successfully!")

Loading tokenizer and model...


NameError: name 'models_cache_dir' is not defined

In [6]:
# Basic response generation function (without conversation history)
def basic_response(prompt, max_tokens=512, temperature=0.7):
    """Generate a response from the model
    
    Usage:
    prompt = "Why is the sky blue?"
    response = generate_response(prompt)
    print(f"Question: {prompt}")
    print(f"\nAnswer: {response}")
    """

    # Format as chat message
    messages = [{"role": "user", "content": prompt}]

    # Apply chat template
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    # Tokenize
    inputs = tokenizer([text], return_tensors="pt").to(model.device)

    # Generate
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_tokens,
        temperature=temperature,
        do_sample=True,
        top_p=0.9
    )

    # Decode
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract just the assistant's response
    response = generated_text.split("assistant")[-1].strip()

    return response

# Chat functionality with full conversation history
def chat_with_model(message, history):
    """Chat with full conversation history"""
    
    # Build messages from history
    messages = []
    for user_msg, assistant_msg in history:
        messages.append({"role": "user", "content": user_msg})
        messages.append({"role": "assistant", "content": assistant_msg})

    messages.append({"role": "user", "content": message})
    
    # Apply chat template
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    
    inputs = tokenizer([text], return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=512,
        temperature=0.7,
        do_sample=True,
        top_p=0.9
    )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = response.split("assistant")[-1].strip()
    
    return response

In [9]:
# Close any existing Gradio instances
gr.close_all()

# Create Gradio interface
demo = gr.ChatInterface(
    fn=chat_with_model, # Run the chat_with_model function
    title="Chatbot Application",
    description="Chat with Qwen Model"
)

demo.launch()

e:\Koding dan Kecerdasan Artificial\.venv\Lib\site-packages\gradio\chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7860


ReadError: [WinError 10054] An existing connection was forcibly closed by the remote host

### llama.cpp

In [ ]:
# Cell 1: Install llama-cpp-python with GPU support using uv
# This compiles with CUDA support for GPU acceleration
!CMAKE_ARGS="-DGGML_CUDA=on" uv pip install llama-cpp-python

print("✓ llama-cpp-python installed with GPU support!")

In [3]:
# Verify installation
try:
    import llama_cpp
    print(f"✓ llama-cpp version: {llama_cpp.__version__}")
except ImportError:
    print("✗ Installation failed. Try running this cell again.")

✓ llama-cpp version: 0.3.16


In [ ]:
# Cell 2: Download a GGUF model
# Using huggingface-cli to download models easily
!uv pip install huggingface-hub

In [ ]:
from huggingface_hub import hf_hub_download
import os

# Example: Download a small quantized model
# You can change this to any GGUF model from Hugging Face
model_repo = "bartowski/Qwen2.5-0.5B-Instruct-GGUF"
model_file = "Qwen2.5-0.5B-Instruct-Q4_K_M.gguf"  # Q4 quantization = good balance

# Set cache directory to local models folder
models_cache_dir = "./models"

print(f"Downloading {model_file} from {model_repo}...")
model_path = hf_hub_download(
    repo_id=model_repo,
    filename=model_file,
    cache_dir=models_cache_dir
)

print(f"✓ Model downloaded to: {model_path}")

e:\Koding dan Kecerdasan Artificial\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


✓ Model downloaded to: ./models\models--bartowski--Qwen2.5-0.5B-Instruct-GGUF\snapshots\41ba88dbac95fed2528c92514c131d73eb5a174b\Qwen2.5-0.5B-Instruct-Q4_K_M.gguf


e:\Koding dan Kecerdasan Artificial\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in E:\Koding dan Kecerdasan Artificial\models\models--bartowski--Qwen2.5-0.5B-Instruct-GGUF. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [5]:
# Cell 3: Initialize llama.cpp and test
from llama_cpp import Llama

print("Loading model into llama.cpp...")

llm = Llama(
    model_path=model_path,
    n_ctx=4096,           # Context window size
    n_gpu_layers=-1,      # -1 = offload all layers to GPU, 0 = CPU only
    n_batch=512,          # Batch size for prompt processing
    n_threads=4,          # Number of CPU threads
    verbose=False         # Set to True for debugging
)

print("✓ Model loaded successfully!")

# Test generation
response = llm(
    "Why is the sky blue?",
    max_tokens=256,
    temperature=0.7,
    top_p=0.9,
    stop=["User:", "\n\n\n"]
)

print("\n=== Test Response ===")
print(response['choices'][0]['text'])

Loading model into llama.cpp...


llama_context: n_ctx_per_seq (4096) < n_ctx_train (32768) -- the full capacity of the model will not be utilized


✓ Model loaded successfully!

=== Test Response ===
 This is a very simple question, but one that can be quite challenging to answer. The sky is blue because of the scattering of sunlight by tiny particles in the air, such as water droplets and pollen. These particles absorb some of the light that enters the atmosphere, but some of it is reflected back into space. The remaining light is scattered in all directions, which we see as the sky. The blue color of the sky is due to the fact that the blue color is the color of the particles that absorb the most sunlight. The rest of the light is reflected back into space, which is why the sky appears blue. The color of the sky is also affected by the angle of the sun and the time of day. The sun's position in the sky and the time of day can affect the scattering of light and the color of the sky. Overall, the blue color of the sky is due to the scattering of sunlight by tiny particles in the air, which is caused by the fact that water droplets